<font size = '12' ><b>Data preparetion</b></font>

In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, 'C:/Users/User/Desktop/Week-3/scripts')
data = pd.read_csv('C:/Users/User/Desktop/10/MachineLearningRating_v3/MachineLearningRating_v3.txt', delimiter='|', low_memory=False)

print("\nData Types of Each Column:")
print(data.dtypes)




Data Types of Each Column:
UnderwrittenCoverID           int64
PolicyID                      int64
TransactionMonth             object
IsVATRegistered                bool
Citizenship                  object
LegalType                    object
Title                        object
Language                     object
Bank                         object
AccountType                  object
MaritalStatus                object
Gender                       object
Country                      object
Province                     object
PostalCode                    int64
MainCrestaZone               object
SubCrestaZone                object
ItemType                     object
mmcode                      float64
VehicleType                  object
RegistrationYear              int64
make                         object
Model                        object
Cylinders                   float64
cubiccapacity               float64
kilowatts                   float64
bodytype                     object


<font size = '5'><b>Handling Missing Data</b></font>

In [2]:
# Fill missing categorical data with a placeholder
categorical_cols = ['Bank', 'AccountType', 'VehicleType', 'make', 'Model', 'bodytype', 
                     'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 
                     'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 
                     'TermFrequency', 'ExcessSelected', 'CoverCategory', 'CoverType', 
                     'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType']

data[categorical_cols] = data[categorical_cols].fillna('Unknown')


In [3]:
# Fill missing numerical data with the median
numerical_cols = ['mmcode', 'Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors', 
                   'CustomValueEstimate', 'NumberOfVehiclesInFleet', 'SumInsured', 
                   'CalculatedPremiumPerTerm', 'TotalPremium', 'TotalClaims']

data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].median())


In [4]:
# Convert date columns to datetime format
data['TransactionMonth'] = pd.to_datetime(data['TransactionMonth'], errors='coerce')
data['VehicleIntroDate'] = pd.to_datetime(data['VehicleIntroDate'], errors='coerce')

# Fill missing dates with a placeholder or the mean date
data['TransactionMonth'].fillna(pd.Timestamp('1900-01-01'), inplace=True)
data['VehicleIntroDate'].fillna(pd.Timestamp('1900-01-01'), inplace=True)


<font size = '5'><b>Feature Engineering</b></font>

In [5]:
import numpy as np

# Example of creating new features
data['CustomValuePerDoor'] = data['CustomValueEstimate'] / (data['NumberOfDoors'].replace(0, np.nan))
data['PremiumPerKilowatt'] = data['CalculatedPremiumPerTerm'] / (data['kilowatts'].replace(0, np.nan))
data['ClaimRatio'] = data['TotalClaims'] / (data['TotalPremium'].replace(0, np.nan))

# Handle infinite values or NaN in new features
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(0, inplace=True)


<font size = '5'><b>Encoding Categorical Data</b></font>

In [6]:
# Convert categorical columns to numeric format using one-hot encoding
data_encoded = pd.get_dummies(data, columns=['Bank', 'AccountType', 'VehicleType', 'make', 'Model', 
                                             'bodytype', 'AlarmImmobiliser', 'TrackingDevice', 
                                             'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 
                                             'Rebuilt', 'Converted', 'CrossBorder', 'TermFrequency', 
                                             'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 
                                             'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType'])


<font size = '5'><b>Train-Test Split</b></font>

In [7]:
from sklearn.model_selection import train_test_split

# Define features and target
X = data_encoded.drop(['TotalPremium', 'TotalClaims'], axis=1)
y_premium = data_encoded['TotalPremium']
y_claims = data_encoded['TotalClaims']

# Split the data into training and test sets
X_train, X_test, y_train_premium, y_test_premium = train_test_split(X, y_premium, test_size=0.3, random_state=42)
X_train, X_test, y_train_claims, y_test_claims = train_test_split(X, y_claims, test_size=0.3, random_state=42)


<font size ='10'><b> Modeling Techniques</b></font>

In [9]:
# Extract features from datetime
X_train['Year'] = X_train['TransactionMonth'].dt.year
X_train['Month'] = X_train['TransactionMonth'].dt.month
X_train['Day'] = X_train['TransactionMonth'].dt.day

X_test['Year'] = X_test['TransactionMonth'].dt.year
X_test['Month'] = X_test['TransactionMonth'].dt.month
X_test['Day'] = X_test['TransactionMonth'].dt.day

# Drop the original datetime column if no longer needed
X_train = X_train.drop('TransactionMonth', axis=1)
X_test = X_test.drop('TransactionMonth', axis=1)



In [23]:
print(X_train.dtypes)  # Print the data types of all columns
print(X_train.head())  # Print the first few rows to inspect values

UnderwrittenCoverID                int64
PolicyID                           int64
IsVATRegistered                     bool
Citizenship                        int32
LegalType                          int32
                                   ...  
StatutoryClass_Commercial          uint8
StatutoryRiskType_IFRS Constant    uint8
Year                               int64
Month                              int64
Day                                int64
Length: 1296, dtype: object
        UnderwrittenCoverID  PolicyID  IsVATRegistered  Citizenship  \
584894               152730     12988            False            0   
684893               112490      7685            False            0   
342633               210785     13067            False            0   
550864                45222      2597            False            0   
990309               119649      6833            False            2   

        LegalType  Title  Language  MaritalStatus  Gender  Country  ...  \
584894          1  

In [24]:
X_train = X_train.apply(pd.to_numeric, errors='coerce')  # Convert all columns to numeric, setting invalid parsing to NaN


In [25]:
X_train = X_train.fillna(X_train.mean())  # Fill missing values with column mean
y_train_premium = y_train_premium.fillna(y_train_premium.mean())  # Fill missing values in target variable


In [26]:
print("X_train shape:", X_train.shape)
print("y_train_premium shape:", y_train_premium.shape)


X_train shape: (700068, 1296)
y_train_premium shape: (700068,)


In [27]:
X_train = X_train.select_dtypes(include=[np.number])  # Keep only numeric columns

In [36]:
import warnings
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

warnings.simplefilter(action='ignore', category=FutureWarning)

# Load a sample of your data
data_subset = data.sample(n=10000, random_state=1)  # Adjust the sample size as needed

# Define features and target
X_subset = data_subset.drop(columns=['TotalPremium'])  
y_subset_premium = data_subset['TotalPremium']  

# Check for and handle missing values
X_subset = X_subset.fillna(X_subset.mean())
y_subset_premium = y_subset_premium.fillna(y_subset_premium.mean())

# Ensure data types are numeric
X_subset = X_subset.apply(pd.to_numeric, errors='coerce')
y_subset_premium = pd.to_numeric(y_subset_premium, errors='coerce')

# Convert to numpy arrays if needed
X_subset = X_subset.to_numpy()
y_subset_premium = y_subset_premium.to_numpy()

# Split the subset into training and test sets
X_train_subset, X_test_subset, y_train_premium, y_test_premium = train_test_split(
    X_subset, y_subset_premium, test_size=0.2, random_state=42
)

# Check shapes
print(f'X_train_subset shape: {X_train_subset.shape}')
print(f'y_train_premium shape: {y_train_premium.shape}')
print(f'X_test_subset shape: {X_test_subset.shape}')
print(f'y_test_premium shape: {y_test_premium.shape}')

# Linear Regression for TotalPremium
model_lr_premium = LinearRegression()
model_lr_premium.fit(X_train_subset, y_train_premium)
y_pred_lr_premium = model_lr_premium.predict(X_test_subset)
print(f'Linear Regression (Premium) RMSE: {np.sqrt(mean_squared_error(y_test_premium, y_pred_lr_premium))}')
print(f'Linear Regression (Premium) R^2: {r2_score(y_test_premium, y_pred_lr_premium)}')

# Random Forest for TotalPremium
model_rf_premium = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf_premium.fit(X_train_subset, y_train_premium)
y_pred_rf_premium = model_rf_premium.predict(X_test_subset)
print(f'Random Forest (Premium) RMSE: {np.sqrt(mean_squared_error(y_test_premium, y_pred_rf_premium))}')
print(f'Random Forest (Premium) R^2: {r2_score(y_test_premium, y_pred_rf_premium)}')

# XGBoost for TotalPremium
model_xgb_premium = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model_xgb_premium.fit(X_train_subset, y_train_premium)
y_pred_xgb_premium = model_xgb_premium.predict(X_test_subset)
print(f'XGBoost (Premium) RMSE: {np.sqrt(mean_squared_error(y_test_premium, y_pred_xgb_premium))}')
print(f'XGBoost (Premium) R^2: {r2_score(y_test_premium, y_pred_xgb_premium)}')

X_train_subset shape: (8000, 1346)
y_train_premium shape: (8000,)
X_test_subset shape: (2000, 1346)
y_test_premium shape: (2000,)
Linear Regression (Premium) RMSE: 111.54432709126529
Linear Regression (Premium) R^2: 0.5034106298146928
Random Forest (Premium) RMSE: 105.35461114782521
Random Forest (Premium) R^2: 0.5569940694227333
XGBoost (Premium) RMSE: 111.3917519734961
XGBoost (Premium) R^2: 0.5047682132043996
